$\newcommand{\To}{\Rightarrow}$

In [1]:
import os, sys
sys.path.append(os.path.split(os.getcwd())[0])

In [2]:
from kernel.type import TVar, TFun, boolT
from kernel.term import Term, Var, Const
from data import nat
from data.nat import natT, zero, one, plus, times
from logic import basic
from logic import logic
from logic import matcher
from syntax import printer

thy = basic.load_theory('nat')

## Substitution

Substitution on terms is analogous to substitution on types, but more complicated due to the presence of abstractions (lambda terms), and the need to substitute for both type and term variables. Consider the following term:

In [3]:
Ta = TVar("a")
a = Var("a", Ta)
b = Var("b", Ta)
t = Term.mk_equals(a, b)
print(repr(t))

Comb(Comb(Const(equals,'a => 'a => bool),Var(a,'a)),Var(b,'a))


Observe that there are both type and term variables in $t$. The method `subst_type` takes a dictionary of assignments for type variables, and substitutes using this dictionary.

In [4]:
t2 = t.subst_type({"a": natT})
print(repr(t2))

Comb(Comb(Const(equals,nat => nat => bool),Var(a,nat)),Var(b,nat))


Next, we can apply the method `subst`, which takes a dictionary of assignments for (term) variables, and substitutes using this dictionary.

In [5]:
t3 = t2.subst({"a": zero, "b": one})
print(printer.print_term(thy, t3))

0 = 1


We now look at some examples demonstrating how substitution interacts with abstractions. Consider the following term $t$:

In [6]:
x = Var("x", natT)
y = Var("y", natT)
t = plus(Term.mk_abs(x, plus(x, y))(one), x)
print(printer.print_term(thy, t, unicode=True))

(λx. x + y) 1 + x


This term contains variables $x$ and $y$. Let's substitute 3 for $x$ and 5 for $y$:

In [7]:
t2 = t.subst({"x": nat.to_binary(3), "y": nat.to_binary(5)})
print(printer.print_term(thy, t2, unicode=True))

(λx. x + 5) 1 + 3


Observe that both the $x$ at the end and the $y$ inside the lambda term is substituted. However, the $x$ inside the lambda term is not. This is because the latter $x$ is a bound variable, and quite different from the former $x$. In fact, since the name of the bound variable does not matter, the term $t$ is equivalent to $(\lambda z. z + y) 1 + x$. The fact that the name of the bound variable is the same as a variable outside the lambda term is simply a coincidence.

By default, substitution does not perform $\beta$-conversion (evaluation of functions). For example:

In [8]:
two = nat.to_binary(2)
f = Var("f", TFun(natT, natT))
a = Var("a", natT)
t = f(a)
print(printer.print_term(thy, t))

t2 = t.subst({"f": Term.mk_abs(x, plus(x,two)), "a": two})
print(printer.print_term(thy, t2, unicode=True))

f a
(λx. x + 2) 2


To evaluate $f$ after a substitution, one can use `logic.beta_norm` function introduced previously. The function `logic.subst_norm` combines the three operations above: type substitution, term substitution, and $\beta$-normalization. This function takes the pattern to be substituted, and a pair of dictionaries of assignments for type and term variables.

In [9]:
t3 = logic.subst_norm(t, (dict(), {"f": Term.mk_abs(x, plus(x,two)), "a": two}))
print(printer.print_term(thy, t3))

2 + 2


We can also use `logic.subst_norm` to do the example at the beginning of the section:

In [10]:
Ta = TVar("a")
a = Var("a", Ta)
b = Var("b", Ta)
t = Term.mk_equals(a, b)
t2 = logic.subst_norm(t, ({"a": natT}, {"a": zero, "b": one}))
print(printer.print_term(thy, t2))

0 = 1


## Matching

As with types, we can match a pattern (a term containing type and term variables) with a term. Matching a pattern $p$ with a term $t$ determines whether it is possible to instantiate $p$ to $t$, and if it is possible, produces the assignment of type and term variables. Matching of terms is complicated by the presence of two kinds of variables, as well as presence of abstractions (including the possibility of $\beta$-conversion). We start with the discussion with some simple cases, then gradually move to more complex situations.

The basic function for matching is `matcher.first_order_match`. It takes two arguments: the pattern and the term to be matched. It returns a pair (tyinst, inst), where tyinst is the dictionary of type instantiations, and inst is the dictionary of term instantiations. For example:

In [11]:
A = Var("A", boolT)
B = Var("B", boolT)
p = logic.disj(A, B)
print(printer.print_term(thy, p, unicode=True))

x = Var("x", natT)
t = logic.disj(Term.mk_equals(x,zero), Term.mk_equals(x,one))
print(printer.print_term(thy, t, unicode=True))

instsp = matcher.first_order_match(p, t)
print(printer.print_instsp(thy, instsp))

A ∨ B
x = 0 ∨ x = 1
{}, {A: x = 0, B: x = 1}


Note the use of the printing function `print_instsp` for printing the pair of type and term assignments. If type or term assignments need to be printed separately, we use `print_tyinst` and `print_inst` functions:

In [12]:
tyinst, inst = instsp
print(printer.print_tyinst(thy, tyinst))
print(printer.print_inst(thy, inst))

{}
{A: x = 0, B: x = 1}


The output says the term $x = 0 \vee x = 1$ matches the pattern $A \vee B$, where $A$ is assigned to $x = 0$ and $B$ is assigned to $x = 1$. If there is no match, the function `first_order_match` raises an exception. For example:

In [13]:
t2 = logic.conj(Term.mk_equals(x,zero), Term.mk_equals(x,one))
print(printer.print_term(thy, t2, unicode=True))

matcher.first_order_match(p, t2)   # raises MatchException

x = 0 ∧ x = 1


MatchException: 

We now show an example with type variables.

In [14]:
Ta = TVar("a")
a = Var("a", Ta)
b = Var("b", Ta)
p = Term.mk_equals(a, b)

t = Term.mk_equals(zero, one)
print(printer.print_instsp(thy, matcher.first_order_match(p, t)))

{a: nat}, {a: 0, b: 1}


Matching can certainly go inside abstractions:

In [15]:
x = Var("x", Ta)
P = Var("P", TFun(Ta, boolT))
p = Term.mk_all(x, P(x))
print(printer.print_term(thy, p, unicode=True))

n = Var("n", natT)
A = Var("A", TFun(natT, boolT))
t = Term.mk_all(n, A(n))
print(printer.print_term(thy, t, unicode=True))

print(printer.print_instsp(thy, matcher.first_order_match(p, t), unicode=True))

∀x. P x
∀n. A n
{a: nat}, {P: A}


The previous matching worked because the body of the $\forall$ quantifier is precisely a function applied to the bound variable. However, matching still works if the body is a more general predicate of the bound variable:

In [16]:
t2 = Term.mk_all(n, Term.mk_equals(n, zero))
print(printer.print_term(thy, t2, unicode=True))

print(printer.print_instsp(thy, matcher.first_order_match(p, t2), unicode=True))

∀n. n = 0
{a: nat}, {P: λx. x = 0}


In this case, $p$ can be transformed to $t$ only after instantiation as well as $\beta$-normalization. In general, `first_order_match` determines whether there exists `instsp` such that `logic.subst_norm(p, instsp)` is equal to $t$.

## Preview: applying a theorem

Matching and substitution play crucial roles in proofs in higher-order logic, in particular when applying a theorem. We will start the discussion of theorems and proofs in the next section. Here, we give a brief preview of these ideas, using what we have learned so far.

One important class of theorems is identities. For example, the distributivity of multiplication over addition is stated as follows:

In [17]:
x = Var("x", natT)
y = Var("y", natT)
z = Var("z", natT)

distrib_l = Term.mk_equals(times(x,plus(y,z)), plus(times(x,y),times(x,z)))
print(printer.print_term(thy, distrib_l))

x * (y + z) = x * y + x * z


Here $x,y,z$ are variables that can represent arbitrary natural numbers. We would like to apply this identity to rewrite any expression of the form $x\cdot (y+z)$, for example $a\cdot (2a+(b+1))$, which should be rewritten to $a\cdot 2a+a(b+1)$.

The idea for implementing this functionality is as follows. Suppose the term to be rewritten is $t$. First match the left side of the equality with $t$. If the matching succeeds, use the resulting assignments of variables to instantiate the right side of the equality.

In [18]:
def rewrite(prop, t):
    """Rewrite term t using the identity (equality term) prop."""
    instsp = matcher.first_order_match(prop.lhs, t)
    return logic.subst_norm(prop.rhs, instsp)

We now test this function on an example:

In [19]:
a = Var("a", natT)
b = Var("b", natT)
two = nat.to_binary(2)
t = times(a,plus(times(two,a),plus(b,one)))
print("Before: ", printer.print_term(thy, t))

t2 = rewrite(distrib_l, t)
print("After:  ", printer.print_term(thy, t2))

Before:  a * (2 * a + (b + 1))
After:   a * (2 * a) + a * (b + 1)


We have written our first program for proof automation! But before we get too excited - so far all we are doing is manipulating terms. In the next section, we will start the discussion of theorems, which allows us to describe proofs in a more precise language. In particular, we will see how to make sure that the proofs we construct are correct, without trusting the proof automation we wrote ourselves.